# How To Cut
## Utility-scale QAOA

So to date, I have developed Quantum Circuits to purely measure Entanglement in both a two-qubit bell state, and a 100-qubit GHZ state. It's time to actually solve a problem, specifically the Max-Cut optimisation problem. So as before, we are following along with a wonderful IBM tutorial, however, unlike IBM, we will be diving head first into some of the underlying theory and code in more detail. Really so we can start to build up a stronger foundation of knowledge rather than blindingly following the tutorial. So let's get into it. 



